# App 

## Parameters

In [2]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import Button, Layout
from IPython.display import display
import re, json, datetime, argparse, sys, os, ast, shutil
import nbformat as nbf
from Fast_connectCloud import connector
from GoogleDrivePy.google_drive import connect_drive
from GoogleDrivePy.google_platform import connect_cloud_platform

In [3]:
#!pip install --upgrade git+git://github.com/thomaspernet/GoogleDrive-python

# Create Project and Repo

## Create functions

In [4]:
name_project = widgets.Text(
       value="Find an Awesome name",
       description='Project name', layout=Layout(width='50%', height='80px') )

path_src = widgets.Text(
       value="/Users/thomas/Google Drive/Projects/Data_science/GitHub/" \
"Template_project_Github",
       description='path Projects', layout=Layout(width='50%', height='80px') )
path_dest = widgets.Text(
       value='/Users/thomas/Google Drive/Projects/Data_science/GitHub/Repositories',
       description='Destination Projects', layout=Layout(width='50%', height='80px') )
checkbox = widgets.Checkbox(
           description='Create Github repo',value=True)

In [5]:
butt_git = widgets.Button(description='Add Project')
outt_git = widgets.Output()

def on_butt_clicked_git(_):
    print('Begin Create project')
    new_path = os.path.join(path_dest.value, name_project.value)
    os.mkdir(path=new_path)

    def copytree(src, dst, symlinks=False, ignore=None):
        for item in os.listdir(src):
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if os.path.isdir(s):
                shutil.copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    copytree(path_src.value,new_path)

    ### Rename project
    with open(os.path.join(new_path, 'README.md'), "r") as f:
        lines = f.read()

    lines = lines.replace('PROJECTNAME',
                          name_project.value)

    with open(os.path.join(new_path, 'README.md'), 'w') as file:
        file.write(lines)
        
    print('Project Created here: {}'.format(path_dest.value))
    
    if checkbox.value:
        os.chdir(new_path)

    ### Config the hub file
    ### https://hub.github.com/ -> for mac
    ####https://github.com/github/hub/issues/1067#issuecomment-482133220

        os.system("git init")
        os.system("git add .")
        os.system('git commit -m "Create proejct"')
    ### Create repo github
        os.system('hub create')
    ### Oush to GitHub
        push ="git remote add origin https://github.com/thomaspernet/{}".format(
        name_project.value)
        os.system(push)
        os.system("git push -u origin master")
        print('Project {0} added to github. Please open at https://github.com/thomaspernet/{0}'.format(
            name_project.value))

# App Create project

In [6]:
butt_git.on_click(on_butt_clicked_git)
box = widgets.VBox([name_project,
                    path_src,
                    path_dest,
                    checkbox,
                    butt_git,outt_git])
box

Begin Create project
Project Created here: /Users/thomas/Google Drive/Projects/Data_science/GitHub/Repositories
Project Coda_test added to github. Please open at https://github.com/thomaspernet/Coda_test


# Create Notebook Preprocessing

## Input parameters

In [ ]:
# Objective
menu = widgets.Dropdown(
       options=['Connect_source', 'Analytics', 'Studio'],
       value='Connect_source',
       description='Select:')
menu

In [ ]:
# Objective
connector_engine = widgets.Dropdown(
       options=['GS', 'BG'],
       value='GS',
       description='Select:')
connector_engine

In [ ]:
# label
labels = widgets.Text(
       description='Variables label', 
    layout=Layout(width='50%', height='80px'))

In [ ]:
# Date var -> srtudio
date_var = widgets.Text(
       description='Date Variable (studio)', 
    layout=Layout(width='50%', height='80px'))

In [ ]:
# defining some widgets
text = widgets.Text(
       value='My First Notebook',
       description='Notebook Name', layout=Layout(width='50%', height='80px'))

### Create project name

In [ ]:
path = widgets.Text(
       value='/Users/thomas/Google Drive/Projects/Data_science/GitHub/Repositories',
       description='path Projects', layout=Layout(width='50%', height='80px') )

In [ ]:
top = os.listdir(path.value)
list_project = []
for name in top:
    list_project.append(name)
output_prject = widgets.Output()
dropdown_prject = widgets.Dropdown(options =list_project,
                                  description='Project name')

### Create list spreadsheet

In [ ]:
creds = widgets.Text(
       value='/Users/Thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/',
       description='path creds Google', )

In [ ]:
gs = connector.open_connection(online_connection=False,
                               path_credential=creds.value)

service_gd = gs.connect_remote(engine='GS')
service_gcp = gs.connect_remote(engine='GCP')

gdr = connect_drive.connect_drive(service_gd['GoogleDrive'])

gcp = connect_cloud_platform.connect_console(project='valid-pagoda-132423',
                                             service_account=service_gcp['GoogleCloudP'])

In [ ]:
### dropdown spreadsheet
sheetid = '1fB-h1Sbdy7wkUghQTugYKBbvNS5plFCc3TCK8zWe71M'
sheetname = 'data'

spreadsheets = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
sheetName = sheetname,to_dataframe = True)

In [ ]:
### recherche spreadsheet
def findspreadsheet(spreadsheet_name):
    sheetID = gdr.find_file_id(file_name = spreadsheet_name,
                           to_print = False)
    
    list_ = gdr.listSpreadsheet(sheetID= sheetID)
    
    return list_

#### Spreadsheet
def return_spread(theme, module#, fournisseur
             ):
    dic_ = {
        'spreadsheet':theme,
        'sheet':module
    }
    return dic_

# Spreadsheet
unique_spreadsheet = (spreadsheets.loc[lambda x: 
                                 x['Storage'].isin(['GS'])]['Filename']
                .sort_values().to_list())
spreadW = widgets.Dropdown(options=unique_spreadsheet)
init = spreadW.value

### Sheet
sheetW = widgets.Dropdown(options=
                           findspreadsheet(init),
                          description = 'Select Sheet'
                          )
def select_sheet(theme):
    sheetW.options = findspreadsheet(theme)


i = widgets.interactive(select_sheet, theme=spreadW)

#j = widgets.interactive(
#    return_spread,
#    theme=spreadW,
#    module=sheetW,
    #fournisseur=fournisseurtW
#)
butt = widgets.Button(description='Add Spreadsheets')
outt = widgets.Output()
input_datasets = []
sheetnames = []
def on_butt_clicked(_):
    #outt.clear_output()
    with outt:
        dic_ = return_spread(spreadW.value, sheetW.value)
        input_datasets.append(dic_['spreadsheet'])
        sheetnames.append(dic_['sheet'])
        
        display(input_datasets, sheetnames)

In [ ]:
spread_text = widgets.Text(
            value='[""]',
            description='list Spreadsheet', layout=Layout(width='50%', height='80px'))
sheet_text = widgets.Text(
            value='[""]',
            description='list sheet', layout=Layout(width='50%', height='80px'))

### Create list Tables

In [ ]:
### recherche tabel
def findtable(dataset):
    
    list_ = gcp.list_tables(dataset = dataset)['tables']
    
    return list_

#### table
def return_table(theme, module#, fournisseur
             ):
    dic_ = {
        'dataset':theme,
        'table':module
    }
    return dic_

# Theme
unique_dataset = gcp.list_dataset()['Dataset']
datasetsdW = widgets.Dropdown(options=unique_dataset)
init = datasetsdW.value

### Theme: Module

tablesW = widgets.Dropdown(options=
                           findtable(init),
                           description = 'Select Table'
                          )
init2 = tablesW.value


def select_table(theme):
    tablesW.options = findtable(theme)

i = widgets.interactive(select_table, theme=datasetsdW)

#j = widgets.interactive(
#    return_spread,
#    theme=spreadW,
#    module=sheetW,
    #fournisseur=fournisseurtW
#)
butt1 = widgets.Button(description='Add table')
outt1 = widgets.Output()
input_dataset = []
tables = []

def on_butt_clicked1(_):
    #outt.clear_output()
    with outt1:
        dic_ = return_table(datasetsdW.value, tablesW.value)
        input_dataset.append(dic_['dataset'])
        tables.append(dic_['table'])
        
        display(input_dataset, tables)

In [ ]:
dataset_text = widgets.Text(
            value='[""]',
            description='list Dataset', layout=Layout(width='50%', height='80px'))
table_text = widgets.Text(
            value='[""]',
            description='list Table', layout=Layout(width='50%', height='80px'))

### Generate notebook

#### Processing

In [ ]:
def generate_notebook_processing(params):
    """
    """
    
    notebook_name = params['notebook_name']
    project_name = params['project_name']
    input_datasets = params['input_datasets']
    sheetnames = params['sheetnames']
    bigquery_dataset = params['bigquery_dataset']
    destination_engine = params['destination_engine']
    path_ = params['path_processing']
    path_notebook = params['path_notebook_processing']
    project='valid-pagoda-132423'
    username = "thomas"
    
    with open(path_notebook) as f:
        ipynb = json.load(f)

    nb_cells = []
    output_str = ""

# Parse cells in Jupyter notebook template
    for cell in ipynb['cells']:
        cell_type = cell['cell_type']
        source = cell['source']
        nb_cells.append([cell_type, source])

## Change header
    nb_cells[0][1][0] = nb_cells[0][1][0].replace('TITLE', notebook_name)
    now = datetime.datetime.now()
    nb_cells[0][1][2] = nb_cells[0][1][2].replace('XX',
     now.strftime("%Y-%m-%d %H:%M %Z"))

### Add Project name 

    nb_cells[5][1][8] = nb_cells[5][1][8].replace('PROJECTNAME', project_name) ### Pagoda

##### Find if prefix in list of dtaset
###### If dataset found, then remove from list, and make
##### special variable for special treatment
##### So far only one preffix dataset can be added

    regex = r"\*"
    for d in input_datasets:
        matches = re.search(regex, d)
        if matches:
            dataset_suffix = d.replace("*", "")
            input_datasets.remove(d)
        else:
            try:
                if dataset_suffix:
                    pass
            except:
                dataset_suffix = False

## Add data source
## Load dataset
    datasets = gcp.list_dataset()
    bq_contents = []
### To recover the index of the sheetname in sheetname

    for key, value in datasets.items():
        for dataset in value:
            tables = gcp.list_tables(dataset = dataset)
            bq_contents.append(tables)
## Load Buckets
    buckets = gcp.list_bucket()
    gcs_contents = []
    for key, value in buckets.items():
        for bucket in value:
            blob = gcp.list_blob(bucket = bucket)
            gcs_contents.append(blob)

            dic_result = {
                'BQ': {'Filename': []},
                'GCS': {'Filename': []},
                'GS': {'Filename': [],
                       'ID': [],
                       'sheetname':[]},
            }
            sheetindex = 0

            for l_dataset in input_datasets:
                found = False
                ### Search in BQ
                for i in range(0, len(bq_contents)):
                    regex = r'(\w*{0}\w*)'.format(l_dataset)
                    matches = re.search(regex, str(bq_contents[i]))
                    if matches:
					# print(l_dataset)
                        dic_result['BQ']['Filename'].append(l_dataset)
                        found = True
			### Search in GCS
                if found == False:
                    found = False
                    for i in range(0, len(gcs_contents)):
                        regex = r'(\w*{0}\w*)'.format(l_dataset)
                        matches = re.search(regex, str(gcs_contents[i]))
                        if matches:
                            dic_result['GCS']['Filename'].append(l_dataset)
                            found = True

                if found == False:
                    found = False
					### Search in GS
                    ID_gS = gdr.find_file_id(file_name=l_dataset)

                    if sheetindex < len(sheetnames):
                        sheetName = sheetnames[sheetindex]
                        dic_result['GS']['sheetname'].append(sheetName)
                        dic_result['GS']['Filename'].append(l_dataset)
                        dic_result['GS']['ID'].append(ID_gS)
                        sheetindex += 1

### Add destination

    if destination_engine == 'GCP':
        nb_cells[2][1][5] = nb_cells[2][1][5].replace('None',
												  notebook_name + '.gz')
        nb_cells[2][1][6] = nb_cells[2][1][6].replace('None',
												  notebook_name)
    else:
        nb_cells[2][1][4] = nb_cells[2][1][4].replace('None',
												  notebook_name)

### Add load data
    dic_load_data = {
	'BQ':{'markdown': [],
			'code':[]
		   },
	'GCS': {'markdown': [],
			'code':[]
		   },
	'GS':{'markdown': [],
			'code':[]
		   }
}
    template_gcs_load = """
gcp.download_blob(bucket_name = '{0}',
				  destination_blob_name = '{1}',
				  source_file_name = '{2}')

    df_{3} = pd.read_csv('{4}',
					  compression='gzip',
					  header=0,
					  sep=',',
					  quotechar='"',
					  error_bad_lines=False
					  )
    """

    template_gcs_md = """
## Load {0} from Google Cloud Storage

    Feel free to add description about the dataset or any usefull information.

    """

#### GCS
##### Only one file from  GCS?

    if dic_result['GCS']['Filename']:
        for f, file in enumerate(dic_result['GCS']['Filename']):
            filename = dic_result['GCS']['Filename'][f]
            for p, bucket in enumerate(gcs_contents):
                for i, blob in enumerate(bucket['blob']):
                    regex = r'(\w*{0}\w*)'.format(file)
                    matches = re.search(regex, str(blob))
				
                    if matches:
                        df_ = file.split(".")[0]
					#path = gcs_contents[p]['blob'][3].replace(
					#'/' + file, '')
					### get path
                        regex = r"/[^/]*$"
                        path = re.sub(regex, "", blob) 
                        bucket_name  = bucket['Bucket']
                        ggs_cell = template_gcs_load.format(bucket_name,
										 path,
										 filename,
										 df_,
										 filename
												   )
                        ggs_md = template_gcs_md.format(filename)
                        dic_load_data['GCS']['markdown'].append(ggs_md)
                        dic_load_data['GCS']['code'].append(ggs_cell)

        dic_destination_inf = {
            'Bucket':bucket_name,
            'destination_blob_name':path,
            'new_destination_blob': path.replace('Raw_', 'Processed_')
	}
    else:
	#### We don't know where to save the dataset if no table uploaded from GCS
        dic_destination_inf = {
            'Bucket':'NEED TO DEFINE',
            'destination_blob_name':'NEED TO DEFINE',
            'new_destination_blob':  'XXXXX/Processed_'
	}
##### GBQ
    template_gbq_load = """
      query = (
          "SELECT * "
            "FROM {0}.{1} "

        )

    df_{1} = gcp.upload_data_from_bigquery(query = query, location = 'US')
    df_{1}.head()
    """

    template_gbq_md = """
    ## Load {0} from Google Big Query

    Feel free to add description about the dataset or any usefull information.

    """
    if dic_result['BQ']['Filename']:
        for f, file in enumerate(dic_result['BQ']['Filename']):
            filename = dic_result['BQ']['Filename'][f]
#### Note need to search for thr bigquery dataset. Right now, use the predefined one in the argument
            cell_gbq = template_gbq_load.format(bigquery_dataset,
                filename)
            cell_md = template_gbq_md.format(filename)
            dic_load_data['BQ']['markdown'].append(cell_md)
            dic_load_data['BQ']['code'].append(cell_gbq)

##### GS
    template_gs_load = """
    ### Please go here {0}
    ### To change the range

    sheetid = '{1}'
    sheetname = '{2}'

    df_{3} = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
    sheetName = sheetname,
         to_dataframe = True)
    df_{3}.head()
    """

    template_gs_md = """
    ## Load {0} from Google Spreadsheet

    Feel free to add description about the dataset or any usefull information.

    Profiling will be available soon for this dataset

    """
    if dic_result['GS']['ID']:
        for f, iD in enumerate(dic_result['GS']['ID']):
            sheetid = iD
            sheetname = dic_result['GS']['sheetname'][f]
            filename = dic_result['GS']['Filename'][f]
            url_gs = 'https://docs.google.com/spreadsheets/d/{0}'
            url_gs = url_gs.format(sheetid)
            cell_gs = template_gs_load.format(url_gs,
									  sheetid,
									  sheetname,
									  filename)

            cell_md = template_gs_md.format(filename)
            dic_load_data['GS']['markdown'].append(cell_md)
            dic_load_data['GS']['code'].append(cell_gs)

##### Special treatment for prefix
#### 1 find path in GCS with dot. It means the element in
#### gcs_contents is a filename
#### 2 get the path of the blob, ie remove filename from path
#### 3 Drop duplicate path

    if dataset_suffix:
        regex_dot = r"[.]"
        regex_slash = r".*(?=/.)"
        list_blob_dot = []
        for bucket in gcs_contents:
            for blob in bucket['blob']:
                matches = re.search(regex_dot, blob)
                if matches:
                    matches_ = re.search(regex_slash, blob)
                    full_path = '{0}/{1}'
                    full_path = full_path.format(bucket['Bucket'], matches_.group())
                    list_blob_dot.append(full_path)
        list_blob_dot = list(dict.fromkeys(list_blob_dot))

#### We now have a list of paths. We will check which one has the preffix
        l_dataset = []
        for blob in list_blob_dot:
            split_ = blob.split("/",  1)
            bucket = split_[0]
            blob_ = split_[1]
            prefix = '{0}/{1}'.format(blob_, dataset_suffix)
            l_blob =  gcp.list_blob(bucket = bucket, prefix = prefix)
            if l_blob['blob']:
                break
        for d in l_blob['blob']:
            split_ = d.rsplit("/",  1)
            l_dataset.append(split_[1])   

        template_prefix = """
    df_load_data = pd.DataFrame()
    for dataset in {0}:
    gcp.download_blob(bucket_name = '{1}',
                  destination_blob_name = '{2}',
                  source_file_name = dataset)
                  
    df_temp = pd.read_csv(dataset,
                          compression='gzip',
                          header=0,
                          sep=',',
                          quotechar='"',
                          error_bad_lines=False)
    df_load_data = df_load_data.append(df_temp)
    df_load_data.head()
    """

#### Append to the dic

        ggs_cell = template_prefix.format(l_dataset,bucket, blob_)
        ggs_md = template_gcs_md.format(notebook_name)
        dic_load_data['GCS']['markdown'].append(ggs_md)
        dic_load_data['GCS']['code'].append(ggs_cell)

	### If preffix, then override the final destination for GCP
	### If list dataset has both preffix and not prefix for GCP
	### origin, then preffix overide the destination cell

        dic_destination_inf = {
		'Bucket':bucket,
		'destination_blob_name':blob_,
		'new_destination_blob': blob_.replace('Raw_', 'Processed_')
	}

### Add markdown origin
    template_ds = '\n - {0}'
    l_gcs = []
    gs_url = '[{0}](https://docs.google.com/spreadsheets/d/{1})'
    for key, value in dic_result.items():
        if key == 'BQ':
            l_gcs.append('\n ### Big Query Dataset \n')
            for file in value['Filename']:
                cell_gcs = template_ds.format(file)
                l_gcs.append(cell_gcs)
        elif key == 'GCS':
            l_gcs.append('\n ### Google Cloud Storage Dataset \n')
            for file in value['Filename']:
                cell_gcs = template_ds.format(file)
                l_gcs.append(cell_gcs)
            if dataset_suffix:
                cell_gcs = template_ds.format(dataset_suffix)
                l_gcs.append(cell_gcs)
        else:
            l_gcs.append('\n ### Google Spreadsheet Dataset \n')
            for i, file in enumerate(value['Filename']):
                cell_gs = template_ds.format(file)
                sheet_name = value['Filename'][i]
                sheet_ID = value['ID'][i]
                url = gs_url.format(sheet_name, sheet_ID)
                cell_gs = template_ds.format(url)
                l_gcs.append(cell_gs)
    cell_datasource = ' '.join(l_gcs)

    nb_cells[1][1] = cell_datasource

### Change profiling
    nb_cells[9][1][2] = nb_cells[9][1][2].replace("NAME", notebook_name)

### Upload to the Cloud
    template_upload_bqgcs_md = """

### Move to GCS and BigQuery

We move the dataset to the following:

- **bucket**: *{0}*

- **Destination_blob**: *{1}*
- **name**:  *{2}.gz*
- **Dataset**: *{3}*

- **table**: *{2}*

### GCS

We first need to save *{2}* with `.gz` extension locally then we can move it
to GCS
"""

    template_upload_bqgcs = """
### First save locally
df_final.to_csv(
	'{0}.gz',
	sep=',',
	header=True,
	index=False,
	chunksize=100000,
	compression='gzip',
	encoding='utf-8')

### Then upload to GCS
bucket_name = '{1}'
destination_blob_name = '{2}'
source_file_name = '{0}.gz'
gcp.upload_blob(bucket_name, destination_blob_name, source_file_name)

"""

    template_upload_bq = """

### Move to bigquery
bucket_gcs ='{0}/{1}/{2}.gz'
gcp.move_to_bq_autodetect(dataset_name= '{3}',
							 name_table= '{2}',
							 bucket_gcs=bucket_gcs)
"""

    if destination_engine == 'GCP':
        mkdown_output = template_upload_bqgcs_md.format(dic_destination_inf['Bucket'],
							 dic_destination_inf['new_destination_blob'],
							 notebook_name,
							 bigquery_dataset
							)
        code_output_gcs = template_upload_bqgcs.format(notebook_name,
							 dic_destination_inf['Bucket'],
							 dic_destination_inf['new_destination_blob']
							)
        code_output_bq = template_upload_bq.format(dic_destination_inf['Bucket'],
						  dic_destination_inf['new_destination_blob'],
						  notebook_name,
						  bigquery_dataset
						 )
    else:
        pass

#### Generate nb
    nb = nbf.v4.new_notebook()
    cell_left = 'cell_{0}'
    cell_right = '{0}'


    dic_cells={}
    for i, cell in enumerate(nb_cells[0:6]):
        _source = ''.join(cell[1])
        rhs = cell_right.format(_source)
	#lhs = cell_left.format(i)
        if cell[0] == 'markdown':
            dic_cells["markdown_{}".format(i)] = rhs
        elif cell[0] == 'code':
            dic_cells["code_{}".format(i)] = rhs

### GCS
    next_cells_available = len(dic_load_data['GCS']['markdown']) + 6
    for i, markdown in enumerate(dic_load_data['GCS']['markdown']):
        _md = ''.join(markdown)
        code = dic_load_data['GCS']['code'][i]
        _code = ''.join(code)

        rhs_md = cell_right.format(_md)
        rhs_code = cell_right.format(_code)
	#lhs = cell_left.format(i + 3)
        dic_cells["markdown_{}".format(i+5)] = rhs_md
        dic_cells["code_{}".format(i + 6)] = rhs_code

        next_cells_available = next_cells_available + i

### GBQ
    for i, markdown in enumerate(dic_load_data['BQ']['markdown']):
        _md = ''.join(markdown)
        code = dic_load_data['BQ']['code'][i]
        _code = ''.join(code)

        rhs_md = cell_right.format(_md)
        rhs_code = cell_right.format(_code)
	#lhs = cell_left.format(i + 3)
        dic_cells["markdown_{}".format(i+next_cells_available)] = rhs_md
        dic_cells["code_{}".format(i + next_cells_available + 1)] = rhs_code

        next_cells_available = next_cells_available + 1
    next_cells_available = next_cells_available + 3
### GS
    for i, markdown in enumerate(dic_load_data['GS']['markdown']):
        _md = ''.join(markdown)
        code = dic_load_data['GS']['code'][i]
        _code = ''.join(code)

        rhs_md = cell_right.format(_md)
        rhs_code = cell_right.format(_code)
	#lhs = cell_left.format(i + 3)
        dic_cells["markdown_{}".format(i + next_cells_available)] = rhs_md
        dic_cells["code_{}".format(i + next_cells_available + 1)] = rhs_code

        next_cells_available = next_cells_available + 1
    next_cells_available = next_cells_available + 5

### Add step cells
    _source = ''.join(nb_cells[6][1])
    rhs = cell_right.format(_source)
    dic_cells["markdown_{}".format(next_cells_available)] = rhs
    dic_cells["code_{}".format(next_cells_available)] = []
    next_cells_available = next_cells_available + 2

### profiling

    _source = ''.join(nb_cells[8][1])
    rhs = cell_right.format(_source)
    dic_cells["markdown_{}".format(next_cells_available)] = rhs
    next_cells_available = next_cells_available + 1

    _source = ''.join(nb_cells[9][1])
    rhs = cell_right.format(_source)
    dic_cells["code_{}".format(next_cells_available)] = rhs
    next_cells_available = next_cells_available + 1

### upload to cloud
    _source = ''.join(nb_cells[10][1])
    rhs = cell_right.format(_source)
    dic_cells["markdown_{}".format(next_cells_available)] = rhs
    next_cells_available = next_cells_available + 1

    dic_cells["markdown_{}".format(next_cells_available)] = mkdown_output
    dic_cells["code_{}".format(next_cells_available + 1)] = code_output_gcs
    dic_cells["code_{}".format(next_cells_available + 2)] = code_output_bq

### generate nb
    nb['cells'] = []
    regex = r"^markdown"
    for k, v in dic_cells.items():
        if re.search(regex, k):
            nb['cells'].extend([nbf.v4.new_markdown_cell(v)])
        else:
            nb['cells'].extend([nbf.v4.new_code_cell(v)])
    name_nb = notebook_name + '_preprocessing.ipynb'


    #os.chdir(path_)
    nbf.write(nb, name_nb)
    
    print("Notebook script {} generated sucessfully.".format(notebook_name))
    project = ''
    template_bq_input ="https://console.cloud.google.com/bigquery?project={0}&p={0}&d={1}&t={2}&page=table"
    bq_input = template_bq_input.format(project, bigquery_dataset, notebook_name)
    template_gsutil = "gs//{0}/{1}/{2}.gz"
    if dataset_suffix:
        template_gsutil = template_gsutil.format(dic_destination_inf['Bucket'], 
        dic_destination_inf['destination_blob_name'], dataset_suffix)
    else:
        template_gsutil = template_gsutil.format(dic_destination_inf['Bucket'], 
        dic_destination_inf['destination_blob_name'], 'UNKNOWN')

    studio_ ="auto-nb-studio -u '{0}' -c 'GCP' -n '{1}' -p '{2}' -d 'ADD DATE VAR' -m 'US' -t '/PATH CREDENTIAL'"
    studio_ = studio_.format(username, notebook_name, bigquery_dataset)

    print( """
Please, go to the section Inputs_datasource from
    https://coda.io/d/MasterFile-Database_dvfMWDBnHh8/Main_suj76#_lubx_ \n
 paste the following information \n 
 Storage:
 - Storage: GCS
 - Top_level: {0}
 - Path: {1}
 - Filename: {2}.gz
 - Description: Add detailed description
 - Dataset_documentation: If any, add URL
 - Size: Go to console to get the size
 - Status: Closed
 - Source_data:{3}
 - Link_methodology: If any, add URL
 - JupyterStudio: Leave unselect
 - Profiling: Leave unselect \n
 Big Query :
  - Storage: BQ
  - Top_level: {5}
  - Path: None
  - Filename: {2}
  - Source_data:{4}
  - JupyterStudio: Use \n 
 {6} \n
  to create a notebook for the studio, then select the checkbox. Only if Dataset in BigQuery!
 - Profiling: select the checkbox
      """.format(dic_destination_inf['Bucket'], dic_destination_inf['new_destination_blob'],
     notebook_name, template_gsutil, bq_input, bigquery_dataset, studio_)
     )

#### Analytics

In [ ]:
def generate_notebook_analytics(params):
    """
    """
    
    notebook_name = params['notebook_name']
    project_name = params['project_name']
    input_datasets = params['input_datasets'][0] ### Only one dataset, but its a list
    sheetnames = params['sheetnames'][0]### Only one sheet, but its a list
    bigquery_dataset = params['bigquery_dataset']
    destination_engine = params['destination_engine']
    path_ = params['path_analytics']
    path_notebook = params['path_notebook_analytics']
    project='valid-pagoda-132423'
    username = "thomas"
    pathtoken = params['pathtoken']
    connector_en = params['connector']
    labels = params['labels']
    
    with open(path_notebook) as f:
        ipynb = json.load(f)
        
    nb_cells = []
    output_str = ""

# Parse cells in Jupyter notebook template
    for cell in ipynb['cells']:
        cell_type = cell['cell_type']
        source = cell['source']
        nb_cells.append([cell_type, source])
    name_project = project_name.replace(' ', '_')
## 1 Change the header
    nb_cells[0][1][0] = nb_cells[0][1][0].replace('TITLE', name_project)
    now = datetime.datetime.now()
    nb_cells[0][1][2] = nb_cells[0][1][2].replace('XX', now.strftime("%Y-%m-%d %H:%M %Z"))

## 2 change connector
    if connector_en == 'GS':
        nb_cells[4][1][2] = nb_cells[4][1][2].replace("()", "('GS')")
    else:
        nb_cells[4][1][2] = nb_cells[4][1][2].replace("()", "('GCP')")

# Connect to Google: Token is indicated by the user in the command line
    gs = connector.open_connection(online_connection=False,
                               path_credential=pathtoken)

    if connector_en == 'GS':
        service_gd = gs.connect_remote(engine='GS')
        gdr = connect_drive.connect_drive(service_gd['GoogleDrive'])
    else:
        service_gcp = gs.connect_remote(engine='GCP')
        gcp = connect_cloud_platform.connect_console(project=project,
                                                 service_account=service_gcp['GoogleCloudP'])

    if connector_en == 'GS':

    
        sID = gdr.find_file_id(file_name=input_datasets, to_print=True)
        url_spreadsheet = 'https://docs.google.com/spreadsheets/d/' + sID

    

        print('Loading data, be patient')
    ### get n rows and columns
        try:
            nb_cols, n_row = gdr.getRowAndColumns(sheetID = sID,
                     sheetName = sheetnames)  
        except:
            print('Method 1 failed, trying Method 2')
            n_row = gdr.getLatestRow(sID, sheetnames)  
            nb_cols = gdr.getColumnNumber(sID, sheetnames)

        load_data = gdr.upload_data_from_spreadsheet(sheetID = sID,
         sheetName = sheetnames,
         to_dataframe = True)

        load_data = load_data.apply(pd.to_numeric, errors='ignore')

        cell_upload_template = """
from GoogleDrivePy.google_drive import connect_drive
gdr = connect_drive.connect_drive(service['GoogleDrive'])

df_final = gdr.upload_data_from_spreadsheet(sheetID = '{0}',
     sheetName = '{1}',
     to_dataframe = True)

df_final = df_final.apply(pd.to_numeric, errors='ignore')
df_final.head()
    """
        cell_upload = cell_upload_template.format(sID, sheetnames)

    # Change connector cell
        nb_cells[0][1][18] = nb_cells[0][1][18].replace('XXX', input_datasets)
        nb_cells[0][1][18] = nb_cells[0][1][18].replace('HERE', url_spreadsheet)

        l_dtypes = load_data.dtypes.to_list()

    else:

    #### Need to find the dataset from the table
    ### We are supposed to have it ..

        dtasets = gcp.list_dataset()
        for d in dtasets['Dataset']:
            all_tables = gcp.list_tables(dataset = d)['tables']
            if input_datasets in all_tables:
                dataset = d

        query = (
          "SELECT * "
            "FROM {0}.{1} "

        )

        query = query.format(dataset, input_datasets)

        print('Loading data, be patient')
        load_data = gcp.upload_data_from_bigquery(query = query,
         location = 'US')

        l_dtypes = load_data.dtypes.to_list()

        cell_upload_template = """
from GoogleDrivePy.google_platform import connect_cloud_platform
project = '{2}'
gcp = connect_cloud_platform.connect_console(project = project, 
                                             service_account = service['GoogleCloudP'])    
query = (
          "SELECT * "
            "FROM {0}.{1} "

        )

df_final = gcp.upload_data_from_bigquery(query = query, location = 'US')
df_final.head()
    """
    
        cell_upload = cell_upload_template.format(dataset, input_datasets, project[0])
        nb_cells[0][1][18] = nb_cells[0][1][18].replace('XXX', input_datasets)
    
    nb_cells[5][1] = cell_upload

    ### Set if label exists
    if labels:

        if len(labels) != len(l_dtypes):
            print('ERROR: Data source has {0} variables and you input {1} labels'.format(
            len(l_dtypes), len(labels)
    ))
            sys.exit()

    else:
        labels = list(load_data)


    dic_name = {'Variables':  list(load_data),
           'Labels' : labels,
           'Types': l_dtypes}
    df_ = pd.DataFrame(dic_name, index = None)
    df_html = df_.to_html()
    nb_cells[1][1] = df_html

    template_token = '{0}/'

    nb_cells[4][1][1] = nb_cells[4][1][1].replace('pathtoken', template_token.format(pathtoken))

## generate notebook
    nb = nbf.v4.new_notebook()
    cell_left = 'cell_{0}'
    cell_right = '{0}'

    dic_cells={}
    for i, cell in enumerate(nb_cells):
        _source = ''.join(cell[1])
        rhs = cell_right.format(_source)
        lhs = cell_left.format(i)
        if cell[0] == 'markdown':
            dic_cells["markdown_{}".format(i)] = rhs
        elif cell[0] == 'code':
            dic_cells["code_{}".format(i)] = rhs

    nb = nbf.v4.new_notebook()
    nb['cells'] = []
    regex = r"^markdown"
    for k, v in dic_cells.items():
        if re.search(regex, k):
            nb['cells'].extend([nbf.v4.new_markdown_cell(v)])
        else:
            nb['cells'].extend([nbf.v4.new_code_cell(v)])
    name_nb = name_project + '_analysis.ipynb'
    nbf.write(nb, name_nb)
    
    print("Notebook {} generated sucessfully.".format(project_name))

    jup_lab_cl = "jupyter-lab-launcher -u '{0}' -p '{1}'".format(username, project_name)

    print("""
    Please, go to section Inputs_projects from
 https://coda.io/d/MasterFile-Database_dvfMWDBnHh8/Main_suj76#_lubx_ \n
 paste the following information in Input Project_notebook \n 

 - Project_name: {0}
 - DatasetName: {1}
 - JupyterAnalysis: {2}
 - ThirdpartyTool: If AWS Image or any other VM, paste URL

 """.format(project,input_datasets,  jup_lab_cl)
 )

#### Studio

In [ ]:
def generate_notebook_studio(params):
    """
    """
    
    notebook_name = params['notebook_name']
    project_name = params['project_name']
    input_datasets = params['input_datasets'][0] ### Only one dataset, but its a list
    sheetnames = params['sheetnames'][0]### Only one sheet, but its a list
    bigquery_dataset = params['bigquery_dataset']
    destination_engine = params['destination_engine']
    path_ = params['path_analytics']
    path_notebook = params['path_notebook_studio']
    project='valid-pagoda-132423'
    username = "thomas"
    pathtoken = params['pathtoken']
    connector_en = params['connector']
    labels = params['labels']
    date_var = params['date_var']
    
    with open(path_notebook) as f:
        ipynb = json.load(f)
        
    nb_cells = []
    output_str = ""
    
# Parse cells in Jupyter notebook template
    for cell in ipynb['cells']:
        cell_type = cell['cell_type']
        source = cell['source']
        nb_cells.append([cell_type, source])
    name_project = project_name.replace(' ', '_')
## 1 Change the header
    nb_cells[0][1][0] = nb_cells[0][1][0].replace('TITLE', name_project)
    now = datetime.datetime.now()
    nb_cells[0][1][2] = nb_cells[0][1][2].replace('XX', now.strftime("%Y-%m-%d %H:%M %Z"))

## 2 change connector
    if connector_en == 'GS':
        nb_cells[5][1][1] = nb_cells[5][1][1].replace("()", "('GS')")
    else:
        nb_cells[5][1][1] = nb_cells[5][1][1].replace("()", "('GCP')")

## Connect to Google: Token is indicated by the user in the command line
    gs = connector.open_connection(online_connection = False, 
        path_credential = pathtoken)
    if connector_en == 'GS':
        service_gd = gs.connect_remote(engine = 'GS')
        gdr = connect_drive.connect_drive(service_gd['GoogleDrive'])
    else: 
        service_gcp = gs.connect_remote(engine = 'GCP')
        gcp = connect_cloud_platform.connect_console(project = project, 
                                             service_account = service_gcp['GoogleCloudP'])

### Change cell data source, var name and load data

    if connector_en == 'GS':

    
        sID = gdr.find_file_id(file_name=input_datasets, to_print=True)
        url_spreadsheet = 'https://docs.google.com/spreadsheets/d/' + sID

    

        print('Loading data, be patient')
    ### get n rows and columns
        try:
            nb_cols, n_row = gdr.getRowAndColumns(sheetID = sID,
                     sheetName = sheetnames)  
        except:
            print('Method 1 failed, trying Method 2')
            n_row = gdr.getLatestRow(sID, sheetnames)  
            nb_cols = gdr.getColumnNumber(sID, sheetnames)

        load_data = gdr.upload_data_from_spreadsheet(sheetID = sID,
         sheetName = sheetnames,
         to_dataframe = True)

        load_data = load_data.apply(pd.to_numeric, errors='ignore')

        cell_upload_template = """
from GoogleDrivePy.google_drive import connect_drive
gdr = connect_drive.connect_drive(service['GoogleDrive'])

df_final = gdr.upload_data_from_spreadsheet(sheetID = '{0}',
     sheetName = '{1}',
     to_dataframe = True)

df_final = df_final.apply(pd.to_numeric, errors='ignore')
df_final.head()
    """
        cell_upload = cell_upload_template.format(sID, sheetnames)

    # Change connector cell
        nb_cells[6][1][4] = nb_cells[6][1][4].replace('XXX', input_datasets)
        nb_cells[6][1][4] = nb_cells[6][1][4].replace('HERE', url_spreadsheet)

        l_dtypes = load_data.dtypes.to_list()

    else:

    #### Need to find the dataset from the table

        dtasets = gcp.list_dataset()
        for d in dtasets['Dataset']:
            all_tables = gcp.list_tables(dataset = d)['tables']
            if input_datasets in all_tables:
                dataset = d

        query = (
          "SELECT * "
            "FROM {0}.{1} "

        )

        query = query.format(dataset, input_datasets)

        print('Loading data, be patient')
        load_data = gcp.upload_data_from_bigquery(query = query,
         location = 'US')

        l_dtypes = load_data.dtypes.to_list()

        cell_upload_template = """
from GoogleDrivePy.google_platform import connect_cloud_platform
project = '{2}'
gcp = connect_cloud_platform.connect_console(project = project, 
                                             service_account = service['GoogleCloudP'])    
query = (
          "SELECT * "
            "FROM {0}.{1} "

        )

df_final = gcp.upload_data_from_bigquery(query = query, location = 'US')
df_final.head()
    """
        cell_upload = cell_upload_template.format(dataset, input_datasets, project[0])
        nb_cells[6][1][4] = nb_cells[6][1][4].replace('XXX', input_datasets)

    nb_cells[9][1] = cell_upload

### Set if label exists
    if labels:

        if len(labels) != len(l_dtypes):
            print('ERROR: Data source has {0} variables and you input {1} labels'.format(
            len(l_dtypes), len(labels)
    ))
            sys.exit()

    else:
        labels = list(load_data)


    dic_name = {'Variables':  list(load_data),
           'Labels' : labels,
           'Types': l_dtypes}
    df_ = pd.DataFrame(dic_name, index = None)
    df_html = df_.to_html()
    nb_cells[7][1] = df_html

    ## Change Year
    nb_cells[14][1][4] = nb_cells[14][1][4].replace('CHANGE VARIABLE', date_var)

    ## generate notebook
    nb = nbf.v4.new_notebook()
    cell_left = 'cell_{0}'
    cell_right = '{0}'

    dic_cells={}
    for i, cell in enumerate(nb_cells):
        _source = ''.join(cell[1])
        rhs = cell_right.format(_source)
        lhs = cell_left.format(i)
        if cell[0] == 'markdown':
            dic_cells["markdown_{}".format(i)] = rhs
        elif cell[0] == 'code':
            dic_cells["code_{}".format(i)] = rhs

    nb = nbf.v4.new_notebook()
    nb['cells'] = []
    regex = r"^markdown"
    for k, v in dic_cells.items():
        if re.search(regex, k):
            nb['cells'].extend([nbf.v4.new_markdown_cell(v)])
        else:
            nb['cells'].extend([nbf.v4.new_code_cell(v)])
    name_nb = name_project + '_studio.ipynb'
    nbf.write(nb, name_nb)

### Full

In [ ]:
butt_act = widgets.Button(description='Add table')
outt_act = widgets.Output()


def on_butt_clicked_act(_):
    #outt.clear_output()
    
    list_datasets = ast.literal_eval(spread_text.value)
    tables_bq = ast.literal_eval(table_text.value)
    
    list_datasets.extend(tables_bq)
    while("" in list_datasets) : 
        list_datasets.remove("")
    
    dic_ ={
    'notebook_name' : text.value,
    'connector':connector_engine.value,   
    'project_name' : dropdown_prject.value,
    'input_datasets' : list_datasets,
    'sheetnames' : ast.literal_eval(sheet_text.value),
    'bigquery_dataset' : ast.literal_eval(dataset_text.value),
    'destination_engine' : 'GCP',
    'name_nb' :'{}_preprocessing.ipynb'.format(text.value),
    'path_processing':os.path.join(path.value,dropdown_prject.value, 'Data_preprocessing'),
    'path_analytics':os.path.join(path.value,dropdown_prject.value, 'Data_analysis'),    
    'path_studio':os.path.join(path.value,dropdown_prject.value, 'Notebooks_Ready_to_use_studio'),  
    'path_notebook_processing' : os.path.join(path.value,dropdown_prject.value,
                                   'Data_preprocessing',
                                   'Template_preprocessing.ipynb'),
    'path_notebook_analytics':os.path.join(path.value,dropdown_prject.value,
                              'Data_analysis',
                              'Template_analysis.ipynb'
                             ),
    'path_notebook_studio':os.path.join(path.value,dropdown_prject.value,
                              'Notebooks_Ready_to_use_studio',
                              'Template_studio.ipynb'
                             ),
    'pathtoken':creds.value,
     'labels':labels.value,
        'date_var':date_var.value
        
    }
    
    if menu.value == "Connect_source":
        os.chdir(dic_['path_processing'])
        print('Connect_source')
        #print('Begin to create Notebook {}'.format(dic_['notebook_name']))
        #generate_notebook_processing(dic_)
    elif menu.value == "Analytics":
        
        os.chdir(dic_['path_analytics'])
        print("Analytics")
        generate_notebook_analytics(params= dic_)
    else:
        os.chdir(dic_['path_studio'])
        print("Studio")
        generate_notebook_studio(params=dic_)
    

# App Create Notebook

In [ ]:
butt_act.on_click(on_butt_clicked_act)
box = widgets.VBox([menu,connector_engine,text, path, dropdown_prject, creds,
                    spread_text, sheet_text,
                    dataset_text, table_text,date_var,  labels, butt_act,outt_act])
butt.on_click(on_butt_clicked)
# display
data_spread = widgets.VBox([spreadW, sheetW,butt,outt])
butt1.on_click(on_butt_clicked1)
# display
data_table = widgets.VBox([datasetsdW, tablesW,butt1,outt1])
children = [box, data_spread, data_table]
# initializing a tab
tab = widgets.Tab()
# setting the tab windows 
tab.children = children
# changing the title of the first and second window
tab.set_title(0, 'Parameters')
tab.set_title(1, 'Select Spreadsheets')
tab.set_title(2, 'Select Tables')
tab

# Create Transfert Knowledge

## Input parameters

In [ ]:
creds = widgets.Text(
       value='/Users/Thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/',
       description='path creds Google', )

# Date var -> srtudio
excel_path = widgets.Text(
       description='Path to Excel file', 
    layout=Layout(width='50%', height='80px'))

## Prepare function

In [ ]:
##### Function to run the program
def prepare_transfert(nameFile, gdr = None, move_to_drive = False):
    """
    """
    ### Get topic
    #regex = r"\.[^.]*$"
    #topic = re.sub(regex, '', nameFile)
    topic = os.path.splitext(os.path.basename(nameFile))[0]
    
    ### Load excel

    print('Loading {}'.format(topic))
    df = pd.read_excel(nameFile)
    
    ### Add vars
    df['Topic'] = topic
    date_today = pd.Timestamp.today()
    df['Date_added'] = date_today
    df['Date_added'] = df['Date_added'].dt.strftime('%Y-%m-%d')
    
    ### Drop useless vars
    df = df.drop(columns = ['Highlight Color', 'Highlight Color Code'])
    
    ### Reorder
    reorder = ['Topic', 'Date', 'Date_added', 'Website Title',
           'URL', 'Color Category', 'Highlighted Text', 'Note']

    df = df[reorder]

    na_note = df['Note'].isna().sum()
    if na_note != 0:
        df = df.fillna("#{}".format(topic))
        print("""
        There are {} highlights without tags over {}. 
        It is about {:.0%} of the total
        Each empty highlight is filled by #{}
        """.format(na_note,df.shape[0],na_note/df.shape[0], topic))
    
    ### Compare with existing data
    #### Always this spreadsheet
    
    sheetID = '1B5hi8fKckMY15BA3g1a9t7wBtG1IEJiw5mmyKS1Exwc'
    sheetName = 'MetaData'
    df_master = gdr.upload_data_from_spreadsheet(sheetID,
                                                 sheetName,
                                                 to_dataframe = True)
    
    #### Extract only same topic
    
    df_master =  df_master[df_master['Topic'] == topic]
    
    ### Return from df only new highlights
    
    toCompare = df_master['Highlighted Text'].tolist()
    new_highlights = df[~df['Highlighted Text'].isin(toCompare)]
    
    if new_highlights.empty:
        print('The Excel file {0} does not contain new highlights'.format(nameFile))
        exit()
    else:
        n_newHighlights = new_highlights.shape[0]
        print("""
    {} new highlights will be added to MasterFile and Dynalist.
    Feel free to move to Dynalist the new highlights and change the Status to
    Transfered """.format(n_newHighlights))
    #### Reshape to Dynalist
    print('The spreadsheet is availab here \n \
    https://docs.google.com/spreadsheets/d/1B5hi8fKckMY15BA3g1a9t7wBtG1IEJiw5mmyKS1Exwc')
    
    col_needed = ['Highlighted Text' , 'Website Title', 'Note', 'Color Category']
    df_dynalist = new_highlights.copy()
    df_dynalist = df_dynalist[col_needed]
    df_dynalist['ID'] = df_dynalist['Website Title'
                                 ].rank(
    method='dense').astype(int)
    df_dynalist = df_dynalist.set_index(['ID', 'Note', 'Color Category'])
    df_dynalist = df_dynalist.stack().reset_index()
    df_dynalist = df_dynalist.rename(
    columns={
             'level_3' : 'origin',
             0: 'highlights'})
    df_dynalist['Topic'] = topic
    df_dynalist['Date_added'] = date_today
    df_dynalist['Date_added'] = df_dynalist['Date_added'].dt.strftime('%Y-%m-%d')
    df_dynalist['status'] = 'Pending'
    reorder = ['Topic', 'Date_added', 'ID', 'Color Category', 'Note', 'origin',
               'highlights', 'status']
    
    df_dynalist = df_dynalist[reorder]
    
    #### Move to Google Spreadsheet
    if move_to_drive:
        gdr.add_data_to_spreadsheet(data = new_highlights,
                        sheetID = sheetID,
                        sheetName = sheetName,
                        detectRange = True,
                        rangeData = None)
    
        gdr.add_data_to_spreadsheet(data = df_dynalist,
                        sheetID = sheetID,
                        sheetName = 'Transfert_knowledge',
                        detectRange = True,
                        rangeData = None)

In [ ]:
butt_knowledge = widgets.Button(description='Add knowledge')
outt_knowledge = widgets.Output()

In [ ]:
def on_butt_clicked_knowledge(_):
    gs = connector.open_connection(online_connection = False, 
    path_credential = creds.value)

    service_gd = gs.connect_remote(engine = 'GS')
    gdr = connect_drive.connect_drive(service_gd['GoogleDrive'])

    n_newHighlights = prepare_transfert(excel_path.value,
                                        gdr = gdr,
                                        move_to_drive = True)

# App Transfer knowledge

In [ ]:
data_knowledge = widgets.VBox([creds, excel_path,butt_knowledge,outt_knowledge])
butt_knowledge.on_click(on_butt_clicked_knowledge)
data_knowledge